Signature: generate_months(start_date, end_date)
Docstring:
Generate a list of dates corresponding to the first day of each month between two dates.

Args:
start_date (datetime.date): The starting date.
end_date (datetime.date): The ending date.

Returns:
list: A list of date objects for the first day of each month within the specified range.
File:      ~/python/v2.0.0/helperfunctions.py
Type:      function


In [19]:
wef = date(2022,6,1)
til = date(2023,7,1)

In [20]:
generate_months(wef,til)

[datetime.date(2022, 6, 1),
 datetime.date(2022, 7, 1),
 datetime.date(2022, 8, 1),
 datetime.date(2022, 9, 1),
 datetime.date(2022, 10, 1),
 datetime.date(2022, 11, 1),
 datetime.date(2022, 12, 1),
 datetime.date(2023, 1, 1),
 datetime.date(2023, 2, 1),
 datetime.date(2023, 3, 1),
 datetime.date(2023, 4, 1),
 datetime.date(2023, 5, 1),
 datetime.date(2023, 6, 1),
 datetime.date(2023, 7, 1)]